In [21]:
# Imports
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from  langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import getpass
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from duckduckgo_search import DDGS
import requests
import streamlit as st
import pymupdf

import logging
# Set up logging configuration
logging.basicConfig(
    level=logging.WARNING,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('partition_pdf.log'),  # write logs to partition_pdf.log file
        logging.StreamHandler()                    # also print logs to console
    ]
)

load_dotenv()
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")
model = init_chat_model("o3-mini", model_provider="openai")




In [22]:
sensor_name = "MCP9808"

In [23]:
# Find and download sensor datasheet

search_query = f"{sensor_name} datasheet filetype:pdf"
search_results = DDGS().text(search_query)
if search_results:
    datasheet_url = search_results[0]['href']
    print(f"Datasheet URL: {datasheet_url}")
    print("Downloading datasheet...")
    response = requests.get(datasheet_url)
    if response.status_code == 200:
        if not os.path.exists(f"/home/steven/FYP/v2_LLM_OS/LLM/Datasheet_DB/{sensor_name}.pdf"):     
            with open(f"/home/steven/FYP/v2_LLM_OS/LLM/Datasheet_DB/{sensor_name}.pdf", "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print("Datasheet downloaded!")
        else:
            print("Datasheet already exists in the database.")
    print("Loading datasheet...")
    datasheet_path = f"/home/steven/FYP/v2_LLM_OS/LLM/Datasheet_DB/{sensor_name}.pdf"
    print("Datasheet loaded!")
else:
    print("No datasheet found for this I2C sensor.")

Datasheet URL: https://cdn-shop.adafruit.com/datasheets/MCP9808.pdf
Datasheet already exists in the database.
Loading datasheet...
Datasheet loaded!


In [24]:
# Load and partition the datasheet into elements
# 5 levels of partitioning
import pymupdf4llm
import pathlib
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

md_path = f"/home/steven/FYP/v2_LLM_OS/LLM/MD_DB/md_{sensor_name}.md"
if not os.path.exists(md_path):
    md_text = pymupdf4llm.to_markdown(datasheet_path)
    pathlib.Path(md_path).write_bytes(md_text.encode())
    print("Datasheet Partition does not exist. Created a new parition")
else:
    md_text = pathlib.Path(md_path).read_text()
    print("Datasheet partition exists. Loaded from local file")

splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=100)

docs = splitter.create_documents([md_text])

print(len(docs))
# Join all document contents into one string
all_text = "\n\n---------XXXX----------\n\n".join(doc.page_content for doc in docs)

# Save to a single file
output_file = f"/home/steven/FYP/v2_LLM_OS/LLM/MD_DB/split_md_{sensor_name}.md"
with open(output_file, "w", encoding="utf-8") as f:
    f.write(all_text)



Datasheet partition exists. Loaded from local file
213


In [25]:
# Embed the datasheet chunks using FAISS
#TODO: We might want to use multiple datasheets for the same sensor
embeddings = OpenAIEmbeddings(
    openai_api_key=os.getenv("OPENAI_API_KEY"), 
    model="text-embedding-ada-002"
)

vector_db_path = f"/home/steven/FYP/v2_LLM_OS/LLM/Datasheet_Vector_DB/{sensor_name}"
if not os.path.exists(vector_db_path):
    vector_db = FAISS.from_documents(docs, embeddings)
    vector_db.save_local(vector_db_path)
    print("Vector DB not found, created and saved a new Vector DB")
else:
    vector_db = FAISS.load_local(vector_db_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector DB found, loaded from local file")

Vector DB found, loaded from local file


In [26]:
#Take 10 most similar chunks from the vector DB using cosine simlarity.
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
query = "Initialize registers hexadecimal value" 

retrieved_chunk = retriever.invoke(query)

In [27]:
# Iterate through the chunks. Ask the LLM if the chunk is helpful for answering the query. (Chunk validation)
# How do I ask LLM if the chunk is helpful, if not mark the chunk as not helpful and retrieve the next chunk?
validation_prompt = ChatPromptTemplate.from_template(
    """
    You are an assistant that validates if a provided document chunk is helpful in answering the user's query.

    QUERY:
    {query}

    CHUNK:
    {chunk}

    Is this chunk helpful for answering the query? Respond ONLY with 'Yes' or 'No'.
    """
)

validated_chunks = []

# Inspect the retrieved chunks (optional, for debugging purposes)
for idx, chunk in enumerate(retrieved_chunk):
    print(f"Retrieved Chunk {idx+1}: {chunk.page_content}")
    prompt = validation_prompt.format_messages(query=query, chunk=chunk.page_content)
    # print(prompt)
    response = model.invoke(prompt).content.strip().lower()
    print(response)
    if 'yes' in response:
        validated_chunks.append(chunk)
        print("YES. Chunk is helpful, proceeding with the next steps")
    else:
        print("NO. Chunk not helpful, moving to next chunk")
        continue

Retrieved Chunk 1: |Col1|Col2|(See Section 5.3 “Summary of Power-on Default” for Power-on Defaults)|Col4|Col5|Col6|Col7|Col8|Col9|Col10|
|---|---|---|---|---|---|---|---|---|---|
|Register Pointer (Hex)|MSB/ LSB|Bit Assignment||||||||
|||7|6|5|4|3|2|1|0|
|0x00|MSB|0|0|0|0|0|0|0|0|
||LSB|0|0|0|1|1|1|1|1|
|0x01|MSB|0|0|0|0|0|Hysteresis||SHDN|
||LSB|Crt Loc|Win Loc|Int Clr|Alt Stat|Alt Cnt|Alt Sel|Alt Pol|Alt Mod|
|0x02|MSB|0|0|0|SIGN|27°C|26°C|25°C|24°C|
||LSB|23°C|22°C|21°C|20°C|2-1°C|2-2°C|0|0|
yes
YES. Chunk is helpful, proceeding with the next steps
Retrieved Chunk 2: |TABLE 5-3: POWER-ON RESET DEFAULTS|Col2|Col3|Col4|
|---|---|---|---|
|Registers||Default Register Data (Hexadecimal)|Power-Up Default Register Description|
|Address (Hexadecimal)|Register Name|||
|0x01|CONFIG|0x0000|Comparator Mode Active-Low Output Alert and Critical Output Output Disabled Alert Not Asserted Interrupt Cleared Alert Limits Unlocked Critical Limit Unlocked Continuous Conversion 0°C Hysteresis|
|0x02|T U

In [28]:
# Consolidate the validated chunks
consolidated_chunks = ""
i = 1
for chunk in validated_chunks:
    consolidated_chunks += f"{i}. {chunk.page_content}\n"
    i += 1
    
print(f"Consolidated Chunks: {consolidated_chunks}")


Consolidated Chunks: 1. |Col1|Col2|(See Section 5.3 “Summary of Power-on Default” for Power-on Defaults)|Col4|Col5|Col6|Col7|Col8|Col9|Col10|
|---|---|---|---|---|---|---|---|---|---|
|Register Pointer (Hex)|MSB/ LSB|Bit Assignment||||||||
|||7|6|5|4|3|2|1|0|
|0x00|MSB|0|0|0|0|0|0|0|0|
||LSB|0|0|0|1|1|1|1|1|
|0x01|MSB|0|0|0|0|0|Hysteresis||SHDN|
||LSB|Crt Loc|Win Loc|Int Clr|Alt Stat|Alt Cnt|Alt Sel|Alt Pol|Alt Mod|
|0x02|MSB|0|0|0|SIGN|27°C|26°C|25°C|24°C|
||LSB|23°C|22°C|21°C|20°C|2-1°C|2-2°C|0|0|
2. |TABLE 5-3: POWER-ON RESET DEFAULTS|Col2|Col3|Col4|
|---|---|---|---|
|Registers||Default Register Data (Hexadecimal)|Power-Up Default Register Description|
|Address (Hexadecimal)|Register Name|||
|0x01|CONFIG|0x0000|Comparator Mode Active-Low Output Alert and Critical Output Output Disabled Alert Not Asserted Interrupt Cleared Alert Limits Unlocked Critical Limit Unlocked Continuous Conversion 0°C Hysteresis|
|0x02|T UPPER|0x0000|0°C|
|0x03|T LOWER|0x0000|0°C|
|0x04|T CRIT|0x0000|0°C|
3

In [29]:
# Chain of Thought Reasoning LLM to extract the I2C address from the consolidated chunks
# https://www.datacamp.com/tutorial/chain-of-thought-prompting

prompt_i2c_template = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant and an expert in I2C Sensors.

    Raw context:
    {chunk}

    From your knowledge, what is the init or initialize registers of {sensor_name}? Show me the reasoning process step by step and use your memory.
    If it is not needed, please say so.
    """
)

prompt_i2c = prompt_i2c_template.format_messages(
    chunk=consolidated_chunks,
    sensor_name=sensor_name
)

response = model.invoke(prompt_i2c).content.strip()
print(f"Response: {response}")

# The context is correct. The output is wrong, but in chatgpt website, the output is correct.
# Maybe they are using reasoning and chain of thought which might be super helpful.

Response: Below is the reasoning process and answer based on the MCP9808’s datasheet and the provided context.

1. The raw context shows tables for registers with names like CONFIG, T UPPER, T LOWER, and T CRIT. These registers correspond to:
 • 0x01 – CONFIG register
 • 0x02 – Temperature upper limit register (T UPPER)
 • 0x03 – Temperature lower limit register (T LOWER)
 • 0x04 – Temperature critical register (T CRIT)

2. The “POWER-ON RESET DEFAULTS” table tells us that when the device powers up (or after a reset):
 – CONFIG (0x01) is 0x0000. This default sets the sensor in continuous conversion mode with alert functions (comparator mode, open-drain output disabled, etc.) in a known “safe” state.
 – T UPPER (0x02), T LOWER (0x03), and T CRIT (0x04) all default to 0x0000, which in the context of temperature thresholds means they are initially set to 0°C.

3. The MCP9808 is designed so that after a power-on, these registers are preset to their known default values. This ensures that i

In [30]:
prompt_i2c_feedback_template = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant and an expert in I2C Sensors.

    My expert told me:
    {i2c_CoT_response}

    What are the hexadecimal values to write to the i2c address to initialize the {sensor_name} sensor?
    If it is not needed, output "INOP".
    Finish the sentence, the hexadecimal values are:
    """
)

prompt_i2c_feedback = prompt_i2c_feedback_template.format_messages(
    i2c_CoT_response=response,
    sensor_name=sensor_name
)
i2c_feedback_response = model.invoke(prompt_i2c_feedback).content.strip()
print(f"Response: {i2c_feedback_response}")

Response: Since the MCP9808’s essential registers are preset to their default values (0x0000) at power-up, there is no need to explicitly write any initialization values if you are using it in its default mode. 

The hexadecimal values are: INOP.


In [31]:
if "INOP" in i2c_feedback_response:
    print("No initialization needed")
else:
    prompt_i2c_cleanup_template = ChatPromptTemplate.from_template(
        """
        You are a helpful assistant and hexadecimal values extractor.

        My expert told me:
        {i2c_feedback_response}

        Extract only the hexadecimal values separated by commas.
        """
    )
    prompt_i2c_cleanup = prompt_i2c_cleanup_template.format_messages(
        i2c_feedback_response=i2c_feedback_response
    )
    i2c_cleanup_response = model.invoke(prompt_i2c_cleanup).content.strip()
    print(f"Response: {i2c_cleanup_response}")

No initialization needed
